#### Ejercicio 7

Escribir un código para estimar la norma infinito de una matriz, usar la fórmula cerrada. Cómo
sería un código si no tenemos una fórmula cerrada? Comparar.

##### Solución

Partiendo de la definición de norma infinito para una matriz a partir de la norma vectorial:

$$
\left\| A \right\|_{\infty} = \sup \left\{ \left\| Ax \right\|_{\infty} : \left\| x \right\|_{\infty} = 1 \right\}
$$

Y aplicando la fórmula cerrada para una matriz 
cuadrada de dimensión $n$ como:

$$
\left\| A \right\|_{\infty} = \max_{1 \le i \le n} \sum_{j=i}^n \left| a_{ij} \right|
$$

Podemos implementarla con la siguiente función `inf_norm_mat()`:

In [1]:
def inf_norm_mat(matrix):
    return max(
        sum(abs(elem) for elem in row) for row in matrix
    )

La verificamos para la matriz ejemplo de las notas de 
clase:

$$
\left( \begin{matrix} 1 & 1 \\ 3 & 0 \end{matrix} \right)
$$

En nuestro caso:

In [2]:
m7 = [[1, 1], [3, 0]]
inf_norm_mat(m7)

3

En el caso de que no fuera posible obtener
la fórmula cerrada de nuestra norma (como sería
para el caso de una norma $p$ en general)
deberíamos aplicar un procedimiento estadístico
generando la mayor cantidad posible de vectores $x$
que posean norma 1. Definimo así una segunda versión
de nuestra función aprovechando que la función 
`matmul` del paquete `numpy` se comporta como
producto escalar si ambos parámetros son vectores
de una dimensión:

In [3]:
from random import randint
from numpy import matmul


def normalize_vec(vector, norm):
    return [elem / norm(vector) for elem in vector]


def inf_norm_vec(vector):
    return max(abs(elem) for elem in vector)


def inf_norm_mat2(matrix, sample_size):
    return max(
        inf_norm_vec(
            matmul(
                matrix, normalize_vec(vector, inf_norm_vec)
            )
        )
        for vector in (
            [
                randint(-100, 100)
                for _ in range(len(matrix[0]))
            ]
            for _ in range(sample_size)
        )
    )

Para determinadas matrices se puede obtener un resultado
aceptable con pocas muestras. Reutilizando nuestra matriz
de ejemplo:

In [4]:
inf_norm_mat2(m7, 10)

3.0

Sin embargo, otras matrices nos obligan a una mayor 
cantidad de muestra para objener un resultado 
cercano al de la fórmula cerrada. Con ayuda de nuestra
función auxiliar `table` que se lista en el Anexo, 
construiremos una tabla comparativa para los 
resultados obtenidos con una matriz en el cálculo
de su norma infinita con diferentes tamaños de 
muestra aleatorias de vectores de norma infinita 1:

In [5]:
from auxiliares import *

tabla(
    [10 ** x for x in range(6)],
    [
        "Tamaño de muestra",
        "$\\begin{Vmatrix}A\\end{Vmatrix}_{\\infty}$",
    ],
    [
        repr,
        lambda x: inf_norm_mat2([[1, -2, 5], [1, 2, 4]], x),
    ],
)

|Tamaño de muestra|$\begin{Vmatrix}A\end{Vmatrix}_{\infty}$|
|:-:|:-:
|1|7.0|
|10|6.460526315789474|
|100|7.941860465116279|
|1000|7.8196721311475414|
|10000|7.930555555555555|
|100000|8.0|


#### Ejercicio 11

Escribir un código para resolver el sistema matricial $Ax = b$ donde $A$ es una matriz que es una
permutación de una matríz triangular superior.

##### Solución

Asumiendo que la matríz $A$ es una permutación de una matriz triangular
superior, aplicamos el algoritmo de resolución de una tal matriz para 
un vector de valores independientes, siguiendo un ordenamiento dado 
por una matriz de permutaciones $P$. Es decir, que resolvemos la matriz
triangular superior $P \times A$ para el vector de valores independientes
$P \times b$. Así, la implementación de la función `solve_triu` resulta:

In [6]:
from itertools import takewhile
from functools import reduce


def solve_triu(matrix, values, permut):
    n = len(matrix)
    perm = []
    for row in permut:
        for i, elem in enumerate(row):
            if elem == 1:
                perm.append(i) 
    return reduce(
        lambda r, i: [
            (
                values[perm[i]]
                - matmul(
                    matrix[perm[i]][i + 1 : n],
                    r,
                )
            )
            / matrix[perm[i]][i]
        ]
        + r,
        reversed(range(n)),
        [],
    )

Verifiquemos la función con el siguiente sistema:

$$
    \left(\begin{matrix} 1 & 2 & 3 \\ 0 & 5 & 4 \\ 0 & 0 & 6 \end{matrix}\right) 
    \times 
    \left(\begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix}\right)
    = 
    \left(\begin{matrix} 7 \\ 8 \\ 9 \end{matrix}\right)
$$

En código:

In [7]:
A = [[1,2,3],
     [0,5,4],
     [0,0,6]]
b = [7,8,9]
P = [[1,0,0],
     [0,1,0],
     [0,0,1]]
x = solve_triu(A, b, P)
print(x)

[1.7000000000000002, 0.4, 1.5]


Si nuestra solución es correcta se debe verificar que $Ax = b$:

In [8]:
matmul(A, solve_triu(A, b, P))

array([7., 8., 9.])

#### Ejercicio 14

Usar el proceso de eliminación de Gauss Escalado para encontrar la descomposición $P \cdot A = L \cdot U$ (comparar con Python) y resolver en cada uno de los casos.

$$
    \left(\begin{matrix} -1 & 1 & -4 \\ 2 & 2 & 0 \\ 3 & 3 & 2 \end{matrix}\right) 
    \times 
    \left(\begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix}\right)
    = 
    \left(\begin{matrix} 0 \\ 1 \\ 1/2 \end{matrix}\right)
$$

$$
    \left(\begin{matrix} 1 & 6 & 0 \\ 2 & 1 & 0 \\ 0 & 2 & 1 \end{matrix}\right) 
    \times 
    \left(\begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix}\right)
    = 
    \left(\begin{matrix} 3 \\ 1 \\ 1 \end{matrix}\right)
$$

##### Solución

Primero definimos nuestra implementación de descomposición $LU$ 
mediante el método de Gauss Escalado para una matriz cuadrada, de manera que nuestra 
función `lu` devolverá una matriz de permutaciones $P$ y una matriz 
que corresponde a la suma matriz triangular superior $U$ (sin permutar) 
más una matriz triangular inferior $L$ (sin permutar) a la que se le
ha restado la matriz identidad (la diagonal contiene lo elementos de 
la triangular superior y no los 1 de la identidad, estos se asumiran 
en los algoritmos de solución). Es decir que para mejor aprovechamiento
de memoria resolveremos la descomposición _in situ_:

In [9]:
def lu(matrix):
    n = len(matrix)
    permut = []
    rows = set(range(n))
    maxim = [max(row, key=abs) for row in matrix]
    for i in range(n):
        pivot = max(
            rows,
            key=lambda j: abs(matrix[j][i] / maxim[j]),
        )
        rows.remove(pivot)
        permut.append(pivot)
        for k in rows:
            matrix[k][i] /= matrix[pivot][i]
            for l in range(i + 1, n):
                matrix[k][l] -= (
                    matrix[k][i] * matrix[pivot][l]
                )
    return (
        [
            [1 if i == p else 0 for i in range(n)]
            for p in permut
        ],
        matrix
    )

De manera que si la intención no es resolver un
sistema sino presentar las matrices correspondientes,
podremos utilizar funciones de copias de matrices
del paquete `numpy`. En nuestro caso:

In [10]:
from numpy import triu, tril, identity


def split_lu(matrix):
    P, G1 = lu(matrix)
    G2 = matmul(P, G1)
    return P, tril(G2, -1) + identity(len(G2)), triu(G2)

También debemos recordar que al procesar las matrices
_in situ_ las mismas se verán modificadas en la siguiente
invocación, por lo que para reutilizarlas en las verificaciones
las copiaremos mediante las funciones del módulo `copy`

In [11]:
from copy import deepcopy

Definamos los parámetros del ejercicio:

In [12]:
m14_1 = [[ -1, 1, -4],[ 2, 2, 0],[ 3, 3, 2]]
b14_1 = [ 0, 1, 1/2]

Compararemos nuestros resultados con los de la implementación de
la descomposición que realiza el paquete `scipy` de Python:

In [13]:
from scipy.linalg import lu as lu2
P1, L1, U1 = split_lu(deepcopy(m14_1))
P2, L2, U2 = lu2(deepcopy(m14_1))

In [14]:
for t in [("nuestro método", P1, L1, U1), ("la librería", P2, L2, U2)]:
    print("Resultados con " + t[0] + ":",
        "\tLa matriz de permutaciones es",
        t[1], 
        "\tLa matriz triangular inferior L es",
        t[2], 
        "\tLa matriz triangular superior U es",
        t[3], sep="\n\n", end="\n\n"
    )

Resultados con nuestro método:

	La matriz de permutaciones es

[[0, 1, 0], [1, 0, 0], [0, 0, 1]]

	La matriz triangular inferior L es

[[ 1.   0.   0. ]
 [-0.5  1.   0. ]
 [ 1.5  0.   1. ]]

	La matriz triangular superior U es

[[ 2.  2.  0.]
 [ 0.  2. -4.]
 [ 0.  0.  2.]]

Resultados con la librería:

	La matriz de permutaciones es

[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]

	La matriz triangular inferior L es

[[ 1.          0.          0.        ]
 [-0.33333333  1.          0.        ]
 [ 0.66666667  0.          1.        ]]

	La matriz triangular superior U es

[[ 3.          3.          2.        ]
 [ 0.          2.         -3.33333333]
 [ 0.          0.         -1.33333333]]



Como podemos observar, las descomposiciones
son diferentes, lo que es esperable ya que
también difieren los pivotes elegidos. Puesto
que si existe una descomposición $LU$ entonces
existen infinitas, debemos verificar que ambas 
cumplan con la igualdad $A = P^{-1}LU$, teniendo
en cuenta que la inversa de la matriz permutaciones
es la propia matriz de permutaciones:

In [15]:
print("Con nuestra descomposición, P * L * U es\n")
print(matmul(P1, matmul(L1,U1)), end="\n\n")
print("Con librería, P * L * U es\n")
print(matmul(P2, matmul(L2,U2)))

Con nuestra descomposición, P * L * U es

[[-1.  1. -4.]
 [ 2.  2.  0.]
 [ 3.  3.  2.]]

Con librería, P * L * U es

[[-1.  1. -4.]
 [ 2.  2.  0.]
 [ 3.  3.  2.]]


Para resolver nuestros sistemas sólo nos falta resolver
el caso de una matriz triangular inferior. Su algoritmo
es muy simular al de la matriz triangular superior con
la salvedad de que en este caso asumiremos que la 
diagonal es la identidad a menos que se indique lo
contrario, mecanismo habitual para aprovechar la 
descomposición _in situ_. Veamos su implementación:

In [16]:
def solve_tril(matrix, values, permut, identity=True):
    n = len(matrix)
    perm = []
    for row in permut:
        for i, elem in enumerate(row):
            if elem == 1:
                perm.append(i) 
    return reduce(
        lambda r, i: r + [
            (
                values[perm[i]]
                - matmul(
                    matrix[perm[i]][:i],
                    r,
                )
            )
            / (1 if identity else matrix[perm[i]][i])
        ],
        range(n),
        [],
    )

Verifiquemos su funcionamiento con un ejemplo
en espejo al utilizado al verificar el 
algoritmo para solucionar la matriz triangular superior:

$$
    \left(\begin{matrix} 6 & 0 & 0 \\ 4 & 5 & 0 \\ 3 & 2 & 1 \end{matrix}\right) 
    \times 
    \left(\begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix}\right)
    = 
    \left(\begin{matrix} 9 \\ 8 \\ 7 \end{matrix}\right)
$$

En código:

In [17]:
A = [[6,0,0],
     [4,5,0],
     [3,2,1]]
b = [9,8,7]
P = [[1,0,0],
     [0,1,0],
     [0,0,1]]
solve_tril(A, b, P, identity=False)

[1.5, 0.4, 1.7000000000000002]

Si nuestra solución es correcta se debe verificar que $Ax = b$. 

In [18]:
matmul(A, solve_tril(A, b, P, identity=False))

array([9., 8., 7.])

Ahora estamos en condiciones de resolver nuestro
primer sistema:

$$
    \left(\begin{matrix} -1 & 1 & -4 \\ 2 & 2 & 0 \\ 3 & 3 & 2 \end{matrix}\right) 
    \times 
    \left(\begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix}\right)
    = 
    \left(\begin{matrix} 0 \\ 1 \\ 1/2 \end{matrix}\right)
$$

En código:

In [19]:
P, L, U = split_lu(deepcopy(m14_1))
I = identity(3)
x = solve_triu(U, solve_tril(L, matmul(P,deepcopy(b14_1)), I), I)
print(x)

[1.25, -0.75, -0.5]


Verifiquemos el resultado ya que $Ax = b $:

In [20]:
matmul([[-1,1,-4],[2,2,0],[3,3,2]], x)

array([0. , 1. , 0.5])

En el caso que queramos trabajar con las matrices
_in situ_, será necesario permutar el resultado
intermedio en vez del vector de valores 
independientes, de modo que:

In [21]:
P, G = lu(deepcopy(m14_1))
x = solve_triu(G, matmul(P, solve_tril(G, deepcopy(b14_1), P)), P)
print(x)

[1.25, -0.75, -0.5]


Usaremos este método para el siguiente sistema:

$$
    \left(\begin{matrix} 1 & 6 & 0 \\ 2 & 1 & 0 \\ 0 & 2 & 1 \end{matrix}\right) 
    \times 
    \left(\begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix}\right)
    = 
    \left(\begin{matrix} 3 \\ 1 \\ 1 \end{matrix}\right)
$$

Lo representamos en el lenguaje:

In [22]:
m14_2 = [[ 1, 6, 0], [ 2, 1, 0], [ 0, 2, 1]]
b14_2 = [ 3, 1, 1]

Lo resolvemos con nuestra implementación:

In [23]:
P, G = lu(deepcopy(m14_2))
x = solve_triu(G, matmul(P, solve_tril(G, deepcopy(b14_2), P)), P)
print(x)

[-0.46590909090909094, 0.6818181818181819, -0.36363636363636365]


Verificamos el resultado:

In [24]:
matmul([[1,6,0],[2,1,0],[0,2,1]], x)

array([ 3.625, -0.25 ,  1.   ])

Como se puede ver, hemos obtenido una solución con
un error considerable. Verifiquemos los resultados
que arroja la librería del lenguaje:

In [25]:
P, L, U = lu2(deepcopy(m14_2))
x = solve_triu(U, solve_tril(L, matmul(P,deepcopy(b14_2)), I), I)
print(x)

[0.2727272727272727, 0.45454545454545453, 0.09090909090909083]


Finalmente, comprobemos cuan cerca de la solución está:

In [26]:
matmul([[1,6,0],[2,1,0],[0,2,1]], x)

array([3., 1., 1.])

#### Ejercicio 15

Calcular los primeros 100 términos del método de Richardson para el sistema

$$
    \left(\begin{matrix} 1 & 1/2 & 1/3 \\ 1/3 & 1 & 1/2 \\ 1/2 & 1/3 & 1 \end{matrix}\right) 
    \times 
    \left(\begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix}\right)
    = 
    \left(\begin{matrix} 11/18 \\ 11/18 \\ 11/18 \end{matrix}\right)
$$

##### Solución

Sabemos que el método de Richardson pertenece
a la familia de métodos iterativos tal que:

$$ Q \times x_{k} = (Q - A) \times x_{k-1} + b $$

En el caso del método de Richardson, $Q = I$, por lo
que nuestra fórmula iterativa resulta:

$$ x_{k} = (I - A) \times x_{k-1} + b $$

Si tomamos arbitrariamente que $x_0 = 0$ entonces $x_1 = b$, que es desde dónde comenzaremos la iteración en
caso de que no se provea una solución inicial. Nuestra implementación resulta:

In [27]:
def richardson(matrix, values, terms, guess=None):
    results = []
    if not guess:
        guess = values
    matrix = identity(len(matrix)) - matrix 
    for _ in range(terms):
        results.append(guess)
        guess = matmul(matrix, guess) + values 
    return results

Expresemos los parámetros del ejercicio en el lenguaje:

In [28]:
m15 = [[ 1, 1/2, 1/3], [ 1/3, 1, 1/2], [ 1/2, 1/3, 1]]
b15 = [ 11/18, 11/18, 11/18]
terms = 100

Resolvemos ahora el ejercicio con el siguiente código:

In [29]:
r = richardson(deepcopy(m15), deepcopy(b15), terms)

Presentamos todos los terminos en la siguiente tabla,
acomodados por filas y luego por columnas:

In [43]:
tabla(
    [map(format_vec, r[i:i + 3]) for i in range(len(r) // 3)],
    ["", "Richardson xn",""]
)

||Richardson xn||
|:-:|:-:|:-:
|[0.611, 0.611, 0.611]|[0.102, 0.102, 0.102]|[0.526, 0.526, 0.526]|
|[0.102, 0.102, 0.102]|[0.526, 0.526, 0.526]|[0.173, 0.173, 0.173]|
|[0.526, 0.526, 0.526]|[0.173, 0.173, 0.173]|[0.467, 0.467, 0.467]|
|[0.173, 0.173, 0.173]|[0.467, 0.467, 0.467]|[0.222, 0.222, 0.222]|
|[0.467, 0.467, 0.467]|[0.222, 0.222, 0.222]|[0.426, 0.426, 0.426]|
|[0.222, 0.222, 0.222]|[0.426, 0.426, 0.426]|[0.256, 0.256, 0.256]|
|[0.426, 0.426, 0.426]|[0.256, 0.256, 0.256]|[0.398, 0.398, 0.398]|
|[0.256, 0.256, 0.256]|[0.398, 0.398, 0.398]|[0.279, 0.279, 0.279]|
|[0.398, 0.398, 0.398]|[0.279, 0.279, 0.279]|[0.378, 0.378, 0.378]|
|[0.279, 0.279, 0.279]|[0.378, 0.378, 0.378]|[0.296, 0.296, 0.296]|
|[0.378, 0.378, 0.378]|[0.296, 0.296, 0.296]|[0.364, 0.364, 0.364]|
|[0.296, 0.296, 0.296]|[0.364, 0.364, 0.364]|[0.307, 0.307, 0.307]|
|[0.364, 0.364, 0.364]|[0.307, 0.307, 0.307]|[0.355, 0.355, 0.355]|
|[0.307, 0.307, 0.307]|[0.355, 0.355, 0.355]|[0.315, 0.315, 0.315]|
|[0.355, 0.355, 0.355]|[0.315, 0.315, 0.315]|[0.348, 0.348, 0.348]|
|[0.315, 0.315, 0.315]|[0.348, 0.348, 0.348]|[0.321, 0.321, 0.321]|
|[0.348, 0.348, 0.348]|[0.321, 0.321, 0.321]|[0.344, 0.344, 0.344]|
|[0.321, 0.321, 0.321]|[0.344, 0.344, 0.344]|[0.325, 0.325, 0.325]|
|[0.344, 0.344, 0.344]|[0.325, 0.325, 0.325]|[0.341, 0.341, 0.341]|
|[0.325, 0.325, 0.325]|[0.341, 0.341, 0.341]|[0.327, 0.327, 0.327]|
|[0.341, 0.341, 0.341]|[0.327, 0.327, 0.327]|[0.338, 0.338, 0.338]|
|[0.327, 0.327, 0.327]|[0.338, 0.338, 0.338]|[0.329, 0.329, 0.329]|
|[0.338, 0.338, 0.338]|[0.329, 0.329, 0.329]|[0.337, 0.337, 0.337]|
|[0.329, 0.329, 0.329]|[0.337, 0.337, 0.337]|[0.330, 0.330, 0.330]|
|[0.337, 0.337, 0.337]|[0.330, 0.330, 0.330]|[0.336, 0.336, 0.336]|
|[0.330, 0.330, 0.330]|[0.336, 0.336, 0.336]|[0.331, 0.331, 0.331]|
|[0.336, 0.336, 0.336]|[0.331, 0.331, 0.331]|[0.335, 0.335, 0.335]|
|[0.331, 0.331, 0.331]|[0.335, 0.335, 0.335]|[0.332, 0.332, 0.332]|
|[0.335, 0.335, 0.335]|[0.332, 0.332, 0.332]|[0.335, 0.335, 0.335]|
|[0.332, 0.332, 0.332]|[0.335, 0.335, 0.335]|[0.332, 0.332, 0.332]|
|[0.335, 0.335, 0.335]|[0.332, 0.332, 0.332]|[0.334, 0.334, 0.334]|
|[0.332, 0.332, 0.332]|[0.334, 0.334, 0.334]|[0.333, 0.333, 0.333]|
|[0.334, 0.334, 0.334]|[0.333, 0.333, 0.333]|[0.334, 0.334, 0.334]|


Comprobemos la exactitud del último término:

In [31]:
matmul(m15, r[-1])

array([0.6111111, 0.6111111, 0.6111111])

#### Ejercicio 16

Escribir un algoritmo para calcular los primeros M pasos del método de Jacobi, y Gauss-Seidel.

##### Solución

Para el caso del método de Jacobi, sabemos que
pertenece a la misma familia que la del método
de Richardson, sólo que en este caso la matriz
$Q$ es la matriz diagonal de $A$ por lo que su 
inversa es fácilmente calculable:

$$ Q^{-1}_{ij} = \left\{ \begin{matrix} 
        \frac{1}{A_{ij}} \quad \text{si} \quad i = j \\
        0 \quad \text{si} \quad i \ne j
                    \end{matrix}
            \right.
$$

Por lo que nuestra fórmula iterativa resulta:

$$ x_{k} = (I - Q^{-1}A) \times x_{k-1} + Q^{-1} \times b $$

Con $G = (I - Q^{-1}A)$ definida como:

$$
G_{ij} = \left\{\begin{matrix}
            0 \quad \text{si} \quad i = j \\
            - a_{ij} / a_{ii} \quad \text{si} \quad i \ne j
        \end{matrix}\right.
$$

Nuestra implementación resulta entonces:

In [32]:
def jacobi(matrix, values, terms, guess=None):
    results = []
    if not guess:
        guess = values
    for i, row in enumerate(matrix):
        diag = row[i]
        values[i] /= diag
        for j, elem in enumerate(row):
            matrix[i][j] = 0 if i == j else - elem / diag
    for _ in range(terms):
        results.append(guess)
        guess = matmul(matrix, guess) + values 
    return results

En el caso del método de Gauss-Seidel, la matriz
$Q$ resulta ser la tringular inferior de $A$, pero
como en este caso el cálculo de su inversa podría
resultar costoso, utilizaremos la fórmula iterativa:

$$ Q \times x_{k} = (Q - A) \times x_{k-1} + b $$

Donde $Q - A$ resulta ser la opuesta aditiva de
la triangular superior de $A$ menos los elementos 
de la diagonal, de manera que
nos queda por resolver un sistema triangular 
inferior. Su implementación:

In [33]:
def gauss_siedel(matrix, values, terms, guess=None):
    results = []
    if not guess:
        guess = values
    trian = tril(matrix)
    matrix = -triu(matrix, 1) 
    for _ in range(terms):
        guess = solve_tril(
            trian, 
            matmul(matrix, guess) + values,
            identity(len(trian)),
            identity=False
        )
        results.append(guess)
    return results

#### Ejercicio 17

Para el siguiente sistema mostrar que tanto Gauss-Seidel como Jacobi convergen para cualquier valor
inicial. Usar el ítem anterior para estimar la solución.

$$
    \left(\begin{matrix} 2 & -1 & 0 \\ 1 & 6 & -2 \\ 4 & -3 & 8 \end{matrix}\right) 
    \times 
    \left(\begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix}\right)
    = 
    \left(\begin{matrix} 2 \\ -4 \\ 5 \end{matrix}\right)
$$

##### Solución

Mostramos a continuación como la solución para 
la matriz del ejercicio converge tanto en el
caso del método de Gauss-Seidel como el de 
Jacobi por ser ésta diagonal dominante.

Recordamos que para ser diagonal dominante basta
que la matriz lo sea por filas o bien por columnas,
y que la difinición establece que el valor
absoluto de del elemento diagonal es mayor a la
sumatoria del valor absoluto del resto de los
elementos (por filas o bien por columnas):

$$ \left| a_{ii} \right| > \sum_{j=1, j \ne i}^n \left| a_{ij}\right| $$

ó

$$ \left| a_{ii} \right| > \sum_{j=1, j \ne i}^n \left| a_{ji}\right| $$

En nuestro caso, por columnas:

$$ \left| a_{11} \right| = 2 > \left| a_{12} \right| + \left| a_{12} \right| = 1 $$

$$ \left| a_{22} \right| = 6 > \left| a_{21} \right| + \left| a_{23} \right| = 3 $$

$$ \left| a_{33} \right| = 8 > \left| a_{31} \right| + \left| a_{32} \right| = 7 $$

Finalmente estimamos la solución para los
primero 11 términos de cada método, con las
funciones implementadas en el ejercicio anterior:

In [34]:
m17 = [[2,-1,0],[1,6,-2],[4,-3,8]]
b17 = [2,-4,5]
terms = 11
j = jacobi(deepcopy(m17), deepcopy(b17), terms)
g = gauss_siedel(deepcopy(m17), deepcopy(b17), terms)

Para analizar la convergencia en ambos métodos
definimos el error relativo de solución con respecto
a los valores indepencientes de la siguiente forma:

$$\frac{\left\|b - Ax_n\right\|_{\infty}}{\left\|b\right\|_{\infty}}$$

In [35]:
def rel_err_vec(observ, expect):
    return (
        inf_norm_vec(
            exp - obs for exp, obs in zip(observ, expect)
        ) 
        / inf_norm_vec(expect) 
    )

Presentamos los términos y los errores relativos en la siguiente tabla:

In [36]:
tabla(
    zip(
        range(11),
        map(format_vec, j),
        map(format_err, [rel_err_vec(matmul(m17, sol), b17) for sol in j]),
        map(format_vec, j),
        map(format_err, [rel_err_vec(matmul(m17, sol), b17) for sol in g])
    ),
    [
        "n",
        "Jacobi $x_n$",
        "Jacobi $\\epsilon$",
        "Gauss-Siedel $x_n$",
        "Gauss-Siedel $\\epsilon$"
    ]
)

|n|Jacobi $x_n$|Jacobi $\epsilon$|Gauss-Siedel $x_n$|Gauss-Siedel $\epsilon$|
|:-:|:-:|:-:|:-:|:-:
|0|[1.000, -0.667, 0.625]|1.200E+00|[1.000, -0.667, 0.625]|1.375E+00|
|1|[0.667, -0.625, -0.125]|2.917E-01|[0.667, -0.625, -0.125]|7.531E-01|
|2|[0.688, -0.819, 0.057]|1.333E-01|[0.688, -0.819, 0.057]|9.925E-02|
|3|[0.590, -0.762, -0.026]|1.122E-01|[0.590, -0.762, -0.026]|6.623E-02|
|4|[0.619, -0.774, 0.044]|2.986E-02|[0.619, -0.774, 0.044]|9.193E-03|
|5|[0.613, -0.755, 0.025]|1.578E-02|[0.613, -0.755, 0.025]|5.693E-03|
|6|[0.622, -0.760, 0.035]|1.059E-02|[0.622, -0.760, 0.035]|8.270E-04|
|7|[0.620, -0.759, 0.029]|3.146E-03|[0.620, -0.759, 0.029]|4.790E-04|
|8|[0.621, -0.760, 0.031]|1.757E-03|[0.621, -0.760, 0.031]|7.256E-05|
|9|[0.620, -0.760, 0.030]|1.014E-03|[0.620, -0.760, 0.030]|3.947E-05|
|10|[0.620, -0.760, 0.030]|3.354E-04|[0.620, -0.760, 0.030]|6.224E-06|


Como se puede observar, el método de Gauss-Siedel, si bien más complejo
computacionalmente, converge más rápidamente que el método de Jacobi.
De todas formas, para el término onceavo, ambos métodos tienen mínimamente 
una precisión de tres cifras decimales.